In [1]:
print("Test!")

Test!


In [2]:
# TODO: Auth file somehow?
import ee
import numpy as np
import geopandas
# import matplotlib.pyplot as plt

# from bqplot import pyplot as plt
# from ipyleaflet import *

# ee.Authenticate();
# ee.Initialize();

import geemap

In [3]:
Map = geemap.Map(center=(40, -100), zoom=4)

In [4]:
shp_path = './Lhasa/Lhasa_RC_DGO2km_updated.shp';
lhasa_shp = geemap.shp_to_ee(shp_path);

In [5]:
Map.addLayer(lhasa_shp, {}, 'Lhasa River SHP');

In [21]:
Map.zoom = 11
Map.center_object(lhasa_shp)

In [36]:
filter = ee.Filter.inList('DGO_FID', [1]);
AOI = lhasa_shp.filter(filter);
filtered_DGO = AOI;
AOI_ID=f'middle_ID{1}';
print(AOI);

START_DATE = '1988-04-06';
END_DATE = '1988-04-08';

CLOUD_FILTER = 80;
START = ee.Date(START_DATE);
END = ee.Date(END_DATE);
DAY_DIFF = ee.Date(END_DATE).difference(START, 'day');
DAY_RANGE = ee.List.sequence(0,DAY_DIFF.subtract(1),1).map(lambda day: START.advance(day,'day'));

LANDSAT_SCALE = 30;
#  /* Working resolution, minimum 30 meters, read
#                          https://developers.google.com/earth-engine/guides/scale
#                          to learn how scales/images pyramids work in GEE */
MIN_PIX = 2; # // To filter small water polygons

eightCo = True;      # // eightConnected boolean for vectorization
nCo = (eightCo+1)*4; #  // NConnected integer for file exports

SIMPLIFY_TOLERANCE_N = 2; 
#/* int or float, multiplied by LANDSAT_SCALE, becomes the tolerance 
#                                  of the simplification of the water polygons */

# Visulization parameters :

NDWI_MIN = -0.3;
NDWI_MAX = 0.3;

NDVI_MIN = -0.5;
NDVI_MAX = 0.5;

CLOUD_PAL = ['gray', 'white'];
NDWI_PAL = ['black','white','blue'];
NDVI_PAL = ['black','white','green'];

# Export parameters :
from datetime import datetime
YMD = datetime.today().strftime('%Y-%m-%d');
FOLDER_NAME = f'GEE_Exports_Lhasa_Landsat_{YMD}';

def getCover(image, AOI, scale):
    totPixels = ee.Number(image.unmask(1).reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    actPixels = ee.Number(image.reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    percCover = actPixels.divide(totPixels).multiply(100).round();
    return image.set('percCover', percCover);

def maskL8sr(image):
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    qa = image.select('QA_PIXEL');
    mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)));
    return image.updateMask(mask);

LANDSAT_FULL = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL'])
    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL'])));


LANDSAT_FULL = (LANDSAT_FULL
                    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                    .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                    .merge(ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))   
                );
                      });

LANDSAT = (LANDSAT_FULL
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER))
    .map(maskL8sr)
    .filterBounds(AOI));

LANDSAT = LANDSAT.map(lambda image: getCover(image, AOI, LANDSAT_SCALE));
LANDSAT = LANDSAT.filterMetadata('percCover', 'greater_than', 98);
def addMNDWI(image):
    return image.addBands(srcImg=image.normalizedDifference(['B2','B5'])).rename('MNDWI'); # B2=greenl B4=SWIR
LANDSAT = LANDSAT.map(addMNDWI);

def preProcess(image):
  image = addMNDWI(image);
  return image.select(['MNDWI']);

def addIS_WATER_MNDWI(image):
    return image.addBands(image.expression('MNDWI >  0.0', {
            'MNDWI': image.select('MNDWI'),
            }).rename('IS_WATER_MNDWI'));

LANDSAT = LANDSAT.map(addIS_WATER_MNDWI);
def IS_WATER_INDIC(INDIC, ImColl):
    return ImColl.select('IS_WATER_' + INDIC).map(lambda image: image.updateMask(image.neq(0)));

IS_WATER_MNDWI = IS_WATER_INDIC('MNDWI', LANDSAT);
Map.addLayer(IS_WATER_MNDWI, {palette:'blue'}, 'water MNDWI', true);

SyntaxError: invalid syntax (<ipython-input-36-6f6e93eab2dc>, line 66)

In [9]:
Map

Map(bottom=3389.0, center=[29.651569731000073, 91.3219064102816], controls=(WidgetControl(options=['position',…